In [1]:
!wget spamham.txt https://raw.githubusercontent.com/vincechan/spam-detection/master/data/SpamDetectionData.txt

--2019-08-25 05:24:11--  http://spamham.txt/
Resolving spamham.txt (spamham.txt)... failed: Name or service not known.
wget: unable to resolve host address ‘spamham.txt’
--2019-08-25 05:24:11--  https://raw.githubusercontent.com/vincechan/spam-detection/master/data/SpamDetectionData.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3602319 (3.4M) [text/plain]
Saving to: ‘SpamDetectionData.txt’

100%[======================================>] 3,602,319   --.-K/s   in 0.1s    

2019-08-25 05:24:12 (35.1 MB/s) - ‘SpamDetectionData.txt’ saved [3602319/3602319]

FINISHED --2019-08-25 05:24:12--
Total wall clock time: 0.5s
Downloaded: 1 files, 3.4M in 0.1s (35.1 MB/s)


In [3]:
import random

def read_file(path):
    """
    read and return all data in a file
    """
    with open(path, 'r') as f:
        return f.read()

def load_data():
    """
    load and return the data in features and labels lists
    each item in features contains the raw email text
    each item in labels is either 1(spam) or 0(ham) and identifies corresponding item in features
    """
    # load all data from file
    data_path = "SpamDetectionData.txt"
    all_data = read_file(data_path)
    
    # split the data into lines, each line is a single sample
    all_lines = all_data.split('\n')

    # each line in the file is a sample and has the following format
    # it begins with either "Spam," or "Ham,", and follows by the actual text of the email
    # e.g. Spam,<p>His honeyed and land....
    
    # extract the feature (email text) and label (spam or ham) from each line
    features = []
    labels = []
    for line in all_lines:
        if line[0:4] == 'Spam':
            labels.append(1)
            features.append(line[5:])
            pass
        elif line[0:3] == 'Ham':
            labels.append(0)
            features.append(line[4:])
            pass
        else:
            # ignore markers, empty lines and other lines that aren't valid sample
            # print('ignore: "{}"'.format(line));
            pass
    
    return features, labels
    
features, labels = load_data()

print("total no. of samples: {}".format(len(labels)))
print("total no. of spam samples: {}".format(labels.count(1)))
print("total no. of ham samples: {}".format(labels.count(0)))

print("\nPrint a random sample for inspection:")
random_idx = random.randint(0, len(labels))
print("example feature: {}".format(features[random_idx][0:]))
print("example label: {} ({})".format(labels[random_idx], 'spam' if labels[random_idx] else 'ham'))

total no. of samples: 2100
total no. of spam samples: 1043
total no. of ham samples: 1057

Print a random sample for inspection:
example feature: <p>Harold prose and from he maddest deem finds will were mine and he from. To spoiled for pile loved deem power he dwelt sadness longed one made long these sister that and oft. One was carnal below in losel. Worse spent for name within now it and deadly and to though have she. He vaunted whence native heralds left in there into in. Parasites reverie day evil dear now favour shamed and come the feud.</p><p>Concubines flaunting uncouth had smile mirthful yes soul a few days the but few carnal had revellers. To partings were and known be paphian woe not he. Condole sighed little glee he weary florid will forgot now seek had all scene time sore deemed that low.</p>
example label: 1 (spam)


In [4]:
from sklearn.model_selection import train_test_split

# load features and labels
features, labels = load_data()

# split data into training / test sets
features_train, features_test, labels_train, labels_test = train_test_split(
    features, 
    labels, 
    test_size=0.2,   # use 10% for testing
    random_state=42)

print("no. of train features: {}".format(len(features_train)))
print("no. of train labels: {}".format(len(labels_train)))
print("no. of test features: {}".format(len(features_test)))
print("no. of test labels: {}".format(len(labels_test)))

no. of train features: 1680
no. of train labels: 1680
no. of test features: 420
no. of test labels: 420


## Bag of words

In [5]:
documents = ['Hello, how are you!',
             'Win money, win from home.',
             'Call me now.',
             'Hello, Call hello you tomorrow?']

lower_case_documents = []
lower_case_documents = [d.lower() for d in documents]
print(lower_case_documents)

['hello, how are you!', 'win money, win from home.', 'call me now.', 'hello, call hello you tomorrow?']


In [6]:
sans_punctuation_documents = []
import string

for i in lower_case_documents:
    sans_punctuation_documents.append(i.translate(str.maketrans("","", string.punctuation)))
    
sans_punctuation_documents

['hello how are you',
 'win money win from home',
 'call me now',
 'hello call hello you tomorrow']

In [7]:
preprocessed_documents = [[w for w in d.split()] for d in sans_punctuation_documents]
preprocessed_documents

[['hello', 'how', 'are', 'you'],
 ['win', 'money', 'win', 'from', 'home'],
 ['call', 'me', 'now'],
 ['hello', 'call', 'hello', 'you', 'tomorrow']]

In [8]:
frequency_list = []
import pprint
from collections import Counter

frequency_list = [Counter(d) for d in preprocessed_documents]
pprint.pprint(frequency_list)

[Counter({'hello': 1, 'how': 1, 'are': 1, 'you': 1}),
 Counter({'win': 2, 'money': 1, 'from': 1, 'home': 1}),
 Counter({'call': 1, 'me': 1, 'now': 1}),
 Counter({'hello': 2, 'call': 1, 'you': 1, 'tomorrow': 1})]


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()

In [10]:
count_vector.fit(documents)
count_vector.get_feature_names()

['are',
 'call',
 'from',
 'hello',
 'home',
 'how',
 'me',
 'money',
 'now',
 'tomorrow',
 'win',
 'you']

In [11]:
doc_array = count_vector.transform(documents).toarray()
doc_array

array([[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 2, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1]])

In [13]:
import pandas as pd
frequency_matrix = pd.DataFrame(doc_array, columns = count_vector.get_feature_names())
frequency_matrix

,are,call,from,hello,home,how,me,money,now,tomorrow,win,you
0,1,0,0,1,0,1,0,0,0,0,0,1
1,0,0,1,0,1,0,0,1,0,0,2,0
2,0,1,0,0,0,0,1,0,1,0,0,0
3,0,1,0,2,0,0,0,0,0,1,0,1


## TFIDF

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
# this is a very toy example, do not try this at home unless you want to understand the usage differences
docs=["the house had a tiny little mouse",
      "the cat saw the mouse",
      "the mouse ran away from the house",
      "the cat finally ate the mouse",
      "the end of the mouse story"
     ]
 
# settings that you use for count vectorizer will go here
tfidf_vectorizer=TfidfVectorizer(use_idf=True)
 
# just send in all your docs here
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(docs)
# get the first vector out (for the first document)
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0]
 
# place tf-idf values in a pandas data frame
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
had,0.493562
little,0.493562
tiny,0.493562
house,0.398203
mouse,0.235185
the,0.235185
ate,0.000000
away,0.000000
cat,0.000000
end,0.000000


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
# this is a very toy example, do not try this at home unless you want to understand the usage differences
docs=["hotel food service",
    "hote food service"
     ]
 
# settings that you use for count vectorizer will go here
tfidf_vectorizer=TfidfVectorizer(use_idf=True)
 
# just send in all your docs here
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(docs)
# get the first vector out (for the first document)
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[1]
 
# place tf-idf values in a pandas data frame
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
hote,0.704909
food,0.501549
service,0.501549
hotel,0.000000


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

# vectorize email text into tfidf matrix
# TfidfVectorizer converts collection of raw documents to a matrix of TF-IDF features.
# It's equivalent to CountVectorizer followed by TfidfTransformer.
vectorizer = TfidfVectorizer(
    input='content',     # input is actual text
    lowercase=True,      # convert to lower case before tokenizing
    stop_words='english' # remove stop words
)
features_train_transformed = vectorizer.fit_transform(features_train)
features_test_transformed  = vectorizer.transform(features_test)

In [37]:
from sklearn.naive_bayes import MultinomialNB
import pickle

def save(vectorizer, classifier):
    '''
    save classifier to disk
    '''
    with open('model.pkl', 'wb') as file:
        pickle.dump((vectorizer, classifier), file)
        
def load():
    '''
    load classifier from disk
    '''
    with open('model.pkl', 'rb') as file:
      vectorizer, classifier = pickle.load(file)
    return vectorizer, classifier

# train a classifier
classifier = MultinomialNB()
classifier.fit(features_train_transformed, labels_train)

# save the trained model
save(vectorizer, classifier)

# score the classifier accuracy
print("classifier accuracy {:.2f}%".format(classifier.score(features_test_transformed, labels_test) * 100))

classifier accuracy 100.00%


In [38]:
import numpy as np
from sklearn import metrics
prediction = classifier.predict(features_test_transformed)
fscore = metrics.f1_score(labels_test, prediction, average='macro')
print("F score {:.2f}".format(fscore))

F score 1.00


In [41]:
vectorizer, classifer = load()

print('\nPerform a test')                    
#email_input = 'enter your email here'
email_input = ["""<p>Register below and join Rajesh Garg (Chief Financial Officer, Landmark Group) and Shail Khiyara (Customer Experience Officer, UiPath) to learn some amazing insights on Landmark Group’s journey into digital transformation through automation, how they addressed the retail industry’s complexity with RPA and find out the top-of-mind CFO challenges that automation can solve quickly.  </p>"""]
email_input_transformed = vectorizer.transform(email_input)
prediction = classifer.predict(email_input_transformed)

print('EMAIL:', email_input)
print('The email is', 'SPAM' if prediction else 'HAM')


Perform a test
EMAIL: ['<p>Register below and join Rajesh Garg (Chief Financial Officer, Landmark Group) and Shail Khiyara (Customer Experience Officer, UiPath) to learn some amazing insights on Landmark Group’s journey into digital transformation through automation, how they addressed the retail industry’s complexity with RPA and find out the top-of-mind CFO challenges that automation can solve quickly.  </p>']
The email is HAM
